In [ ]:
#Import require packages
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [ ]:
#Hyper-parameters
VOCAB_SIZE = 88584
MAXLEN = 250
BATCH_SIZE = 64

#Load the dataset
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

In [ ]:
#Preprocess the dataset
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [ ]:
#Construct LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
#Print model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          2834688   
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Compile the model with loss and optimization
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

#Model training
history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 15s 24ms/step - loss: 0.4308 - acc: 0.8027 - val_loss: 0.3322 - val_acc: 0.8630
Epoch 2/10
625/625 [==============================] - 14s 23ms/step - loss: 0.2370 - acc: 0.9107 - val_loss: 0.3818 - val_acc: 0.8450
Epoch 3/10
625/625 [==============================] - 14s 23ms/step - loss: 0.1829 - acc: 0.9330 - val_loss: 0.3229 - val_acc: 0.8854
Epoch 4/10
625/625 [==============================] - 14s 22ms/step - loss: 0.1558 - acc: 0.9449 - val_loss: 0.3027 - val_acc: 0.8918
Epoch 5/10
625/625 [==============================] - 14s 22ms/step - loss: 0.1319 - acc: 0.9541 - val_loss: 0.3193 - val_acc: 0.8946
Epoch 6/10
625/625 [==============================] - 14s 22ms/step - loss: 0.1144 - acc: 0.9604 - val_loss: 0.3428 - val_acc: 0.8908
Epoch 7/10
625/625 [==============================] - 14s 22ms/step - loss: 0.1015 - acc: 0.9654 - val_loss: 0.4116 - val_acc: 0.8796
Epoch 8/10
625/625 [==============================] - 14s 22ms

In [ ]:
#Evaluate the model using test dataset
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 5s 6ms/step - loss: 0.5051 - acc: 0.8506
[0.5051387548446655, 0.8506399989128113]


In [ ]:
#Perform inference on the pretrained model
word_index = imdb.get_word_index()
#Function to preprocess the user text
def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)

In [ ]:
#Make predictions on the preprocessed text
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

#Some example prompts and predictions
positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

[0.95639265]
[0.42813972]
